# 1) Preparación previa

### Carga de librerías

In [ ]:
import pandas as pd
import re
import numpy as np

### Lectura del dataset original de Properati

In [ ]:
data = pd.read_csv("https://media.githubusercontent.com/media/Agustin-Bulzomi/Projects/main/Programming/Digital%20House%20(Python)/Support%20Files/Project%201/Properati.csv")
data.head()

### Separación de columna con muchas ubicaciones

In [ ]:
# La columna "place_with_parent_names" tiene información separada con '|'. Se separa para obtener info adicional
separado = data["place_with_parent_names"].str.split('|', expand = True)
separado.head()

### Agregado de la nueva información en nuevas columnas

In [ ]:
separado.columns = ['??', 'Pais', 'Zona', 'Partido', 'Barrios', 'Country', 'Otra']
data_concat = pd.concat([data, separado], axis=1)

### Cálculo del tipo de cambio para corroborar

In [ ]:
data_concat['TC'] = data_concat['price_aprox_local_currency'] / data_concat['price_aprox_usd'] 
# Calculando el promedio, decidimos tomar la columna 'price_aprox_usd' como el $ de las propiedades
data_concat['TC'].mean().round(2)

### Revisión de registros nulos según columnas

In [ ]:
data_concat.isnull().sum()

### Creación de función para limpiar superficie

In [ ]:
# La siguiente función nos permite limpiar la superficie según las inconsistencias entre la total y la cubierta.
# Se eligií dicha columna con respecto a 'surface_total_in_m2' ya que tenía menor cantidad de registros nulos
def limpieza_superficie(sup_total, sup_cubierta):
    if sup_total is not None and sup_cubierta is not None:
            if sup_total >= sup_cubierta:
                superficie = sup_total
            else: 
                superficie = np.NaN
    elif sup_total is not None:
            superficie = sup_total
    elif sup_cubierta is not None:
            superficie = sup_cubierta
    else: superficie = np.NaN        
                    
    return superficie

### Aplicación de la susodicha función

In [ ]:
data_concat["superficie"] = data_concat[['surface_total_in_m2','surface_covered_in_m2']].apply(lambda data_concat: limpieza_superficie(data_concat['surface_total_in_m2'],data_concat['surface_covered_in_m2']),axis=1)
data_concat.head(10)

### Creación de la columna de precios por metros cuadrados con la nueva superficie

In [ ]:
data_concat['Precio_USD_por_M2']=data_concat.price_aprox_usd/data_concat.superficie 
data_concat.head()

### Eliminación de columnas

In [ ]:
# Algunas se eliminan por no corresponder con nuestro analisis y otras por estar repetidas
data_eliminacion = data_concat.drop(['Unnamed: 0', 'operation', 'place_with_parent_names', 'place_name', 'country_name', 'state_name',
                                     'description', 'geonames_id', 'lat-lon', 'floor', 'rooms', 'expenses', 'properati_url', 
                                     'image_thumbnail', 'title', '??'], axis = 1)
data_eliminacion.head()

In [ ]:
# Corroboramos el tamaño del resultado
data_eliminacion.shape

### Eliminamos los valores nulos del precio nuevo

In [ ]:
df_final = data_eliminacion[data_eliminacion['Precio_USD_por_M2'].notna()]
df_final

# 2) Análisis estadístico breve del resultado

### Cálculo de % de registros según la provincia/región

In [ ]:
(df_final.Zona.value_counts() / df_final.Zona.size) * 100

### Agrupamos por provincia/región

In [ ]:
data_agrupada_prov = df_final.groupby('Zona')
data_agrupada_prov

### Calculamos las métricas según provincia/región

In [ ]:
medidas = data_agrupada_prov[["Precio_USD_por_M2"]].describe().round(2)

In [ ]:
# Ordenamos según count
medidas.sort_values(by=[('Precio_USD_por_M2', 'count')], ascending = False)

### Métricas generales del data set

In [ ]:
medidas_finales = df_final[["Precio_USD_por_M2"]].describe().round(2)
medidas_finales

### Exportación del DF final

In [ ]:
df_final.to_csv('DF_Final.csv', index = False)